# 🚀 5. AI 剪刀石頭布 (AI Rock Paper Scissors)

在本課程中，你將製作一個完整的互動式 AI 遊戲！

你將學會：
1. **遊戲邏輯**：使用 `if-else` 來判斷誰贏誰輸。
2. **隨機性**：讓電腦隨機出拳。
3. **倒數計時**：使用 `for` 迴圈來製造緊張感。
4. **系統整合**：將相機、AI 和 OLED 螢幕結合成一個遊戲機。

## 練習 1：遊戲規則 (Game Logic)

在使用相機之前，我們必須先教會電腦「輸贏的規則」。

規則如下：
- **石頭 (Rock)** 贏 剪刀 (Scissors)
- **剪刀 (Scissors)** 贏 布 (Paper)
- **布 (Paper)** 贏 石頭 (Rock)

**任務：** 完成下方的 `get_winner` 函式。

In [ ]:
def get_winner(player, computer):
    # 如果出拳一樣，就是平手
    if player == computer:
        return "Draw"
    
    # --- 練習：完成下方的判斷邏輯 ---
    # 提示：使用 'and' 來檢查雙方的出拳
    
    # 情況 1: 玩家出石頭，電腦出剪刀 -> 玩家贏
    elif player == "Rock" and computer == "Scissors":
        return "You Win!"
    
    # 情況 2: 玩家出剪刀，電腦出布 -> 玩家贏
        
    # 情況 3: 玩家出布，電腦出石頭 -> 玩家贏

        
    # 其他情況都是輸
    else:
        return "You Lose!"

# --- 測試你的程式碼 ---
print("測試 1 (石頭 vs 剪刀):", get_winner("Rock", "Scissors")) # 應顯示 You Win!
print("測試 2 (剪刀 vs 石頭):", get_winner("Scissors", "Rock")) # 應顯示 You Lose!

### 💡 解答 (點擊展開)

In [ ]:
def get_winner(player, computer):
    if player == computer:
        return "Draw"
    elif player == "Rock" and computer == "Scissors":
        return "You Win!"
        
    elif player == "Scissors" and computer == "Paper":
        return "You Win!"
        
    elif player == "Paper" and computer == "Rock":
        return "You Win!"
        
    else:
        return "You Lose!"

# --- 測試你的程式碼 ---
print("測試 1 (石頭 vs 剪刀):", get_winner("Rock", "Scissors")) # 應顯示 You Win!
print("測試 2 (剪刀 vs 石頭):", get_winner("Scissors", "Rock")) # 應顯示 You Lose!

## 練習 2：電腦的回合 (Random Choice)

我們不希望電腦每次都只出「石頭」。Python 有一個好用的工具叫做 `random` (隨機)。

**任務：** 使用 `random.choice()` 從清單中隨機選一個拳。

In [ ]:
import random

moves = ["Rock", "Paper", "Scissors"]

# --- 練習：從 moves 清單中隨機選一個 ---
# 提示：random.choice(清單名稱)
computer_move = ________________

print(f"電腦出拳: {computer_move}")

### 💡 解答 (點擊展開)

In [ ]:
import random

moves = ["Rock", "Paper", "Scissors"]

# 解答：
computer_move = random.choice(moves)

print(f"電腦出拳: {computer_move}")

## 練習 3：AI 該不該相信？ (Threshold Logic)

AI 會有看錯的時候。如果它覺得這張照片是「剪刀」，但信心指數 (Confidence) 只有 0.4 (40%)，我們應該忽略它。
此外，如果 AI 看到的是 "Background" (背景)，我們也不該開始倒數。

**任務：** 寫一個函式，只有在 **信心 > 0.8** 且 **不是背景** 時回傳 `True`。

In [ ]:
def is_valid_gesture(label, confidence):
    # 設定門檻值
    CONFIDENCE_THRESHOLD = 0.8
    
    # 請寫出判斷式：
    # 如果 confidence 大於 門檻值 並且 label 不是 "None" 也不是 "Background"
    if __________________________________________________:
        return True
    else:
        return False

# 測試
print("測試 1 (剪刀, 0.9):", is_valid_gesture("Scissors", 0.9)) # 應回傳 True
print("測試 2 (剪刀, 0.5):", is_valid_gesture("Scissors", 0.5)) # 應回傳 False
print("測試 3 (背景, 0.99):", is_valid_gesture("Background", 0.99)) # 應回傳 False

### 💡 解答 (點擊展開)

In [ ]:
def is_valid_gesture(label, confidence):
    CONFIDENCE_THRESHOLD = 0.8
    
    # 判斷：信心度夠高 且 不是背景
    if confidence > CONFIDENCE_THRESHOLD and label not in ["None", "Background"]:
        return True
    else:
        return False

# 測試
print("測試 1 (剪刀, 0.9):", is_valid_gesture("Scissors", 0.9)) # 應回傳 True
print("測試 2 (剪刀, 0.5):", is_valid_gesture("Scissors", 0.5)) # 應回傳 False
print("測試 3 (背景, 0.99):", is_valid_gesture("Background", 0.99)) # 應回傳 False

## 硬體與 AI 設定 (請執行此區塊)

這部分包含了相機驅動、OLED 控制和 AI 模型載入的複雜程式碼。

In [ ]:
import cv2
import time
import os
import random
import numpy as np
from picamera2 import Picamera2
from luma.core.interface.serial import i2c
from luma.core.render import canvas
from luma.oled.device import sh1106
from tflite_runtime.interpreter import Interpreter
import ipywidgets as widgets
from IPython.display import display, Image

# 設定路徑
MODEL_PATH = "/home/pi/rock_paper_scissors/teachable_machine_model/model.tflite"
LABEL_PATH = "/home/pi/rock_paper_scissors/teachable_machine_model/labels.txt"
Target_Size = (224, 224)

# 初始化硬體
def init_hardware():
    oled = None
    interpreter = None
    labels = None
    try:
        serial = i2c(port=1, address=0x3C)
        oled = sh1106(serial)
        oled.clear()
        print("✅ OLED: 連線正常")
    except:
        print("❌ OLED 錯誤")
        return None, None, None

    try:
        interpreter = Interpreter(model_path=MODEL_PATH)
        interpreter.allocate_tensors()
        with open(LABEL_PATH, 'r') as f:
            labels = [line.strip().split()[-1] for line in f.readlines()]
        print("✅ AI 模型: 載入成功")
    except:
        print("❌ 模型錯誤")
        return None, None, None
    return oled, interpreter, labels

# AI 預測工具
def predict_gesture(frame_bgr, interpreter, input_index, output_index, labels):
    ai_input = cv2.resize(frame_bgr, Target_Size)
    ai_input = cv2.cvtColor(ai_input, cv2.COLOR_BGR2RGB)
    input_data = np.expand_dims(ai_input, axis=0)
    interpreter.set_tensor(input_index, input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_index)[0]
    result_index = np.argmax(output_data)
    confidence = output_data[result_index] / 255.0 
    label = labels[result_index]
    return label, confidence

# 更新畫面工具
def update_display(widget, frame_bgr):
    preview_frame = cv2.resize(frame_bgr, (320, 240))
    _, buffer = cv2.imencode('.jpeg', preview_frame, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
    widget.value = buffer.tobytes()

# --- [新增] OLED 顯示工具 (這是我們在第 4 章做好的！) ---
def show_on_oled(device, line1, line2):
    with canvas(device) as draw:
        draw.rectangle(device.bounding_box, outline="white", fill="black")
        draw.text((10, 10), line1, fill="white")
        draw.text((10, 35), line2, fill="white")

print("--- 硬體與工具函式準備完成 ---")

# 🏆 最終挑戰：組裝你的 AI 剪刀石頭布

恭喜你完成了所有的訓練！現在我們手邊有這些零件：

1. **裁判邏輯** (`get_winner`)
2. **電腦隨機出拳** (`random`)
3. **倒數計時技巧** (`for loop`)
4. **AI 過濾器** (`is_valid_gesture`)
5. **OLED 顯示器** (`show_on_oled`)

**現在，請你像工程師一樣，把這些零件組裝進主程式 (`start_game_challenge`)，讓遊戲動起來！**

### 🛠️ 第一步：載入你的工具箱

這裡包含了我們之前寫好的函式。請執行這個區塊，把工具準備好。
*(注意：這裡整合了你在練習 1、4、5 寫好的函式)*

In [ ]:
# 這是你在前面練習中完成的工具，我們把它們整理在這裡
import cv2, time, random
from luma.core.render import canvas

# [練習 1 的成果] 裁判邏輯
def get_winner(player, computer):
    if player == computer: return "Draw"
    elif (player == "Rock" and computer == "Scissors") or \
         (player == "Scissors" and computer == "Paper") or \
         (player == "Paper" and computer == "Rock"):
        return "You Win!"
    else: return "You Lose!"

# [練習 4 的成果] AI 信心過濾
def is_valid_gesture(label, conf):
    return conf > 0.85 and label not in ["None", "Background"]

# [練習 5 的成果] OLED 顯示函式
def show_on_oled(device, line1, line2):
    with canvas(device) as draw:
        draw.rectangle(device.bounding_box, outline="white", fill="black")
        draw.text((10, 10), line1, fill="white")
        draw.text((10, 35), line2, fill="white")

print("✅ 工具箱準備完成！")

### 🎮 第二步：組裝主程式

這是遊戲的核心引擎。請根據註解的提示，將缺少的程式碼補上。
**(請尋找底線 `______` 並填入正確的程式碼)**

In [ ]:
def start_game_challenge():
    # 初始化硬體 
    oled, interpreter, labels = init_hardware()
    if not oled: return
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    in_idx, out_idx = input_details[0]['index'], output_details[0]['index']
    
    print("👇 遊戲畫面將顯示在下方 (按上方停止鈕結束) 👇")
    image_widget = widgets.Image(format='jpeg', width=640, height=480)
    display(image_widget)

    # --- [挑戰 1] 初始化分數變數 ---
    my_score = ______
    pi_score = ______

    # --- 視覺工具 (HUD) ---
    def draw_hud(frame, label, conf, m_score, p_score):
        # 畫出 AI 信心與比分板
        ai_color = (0, 255, 0) if conf > 0.85 else (0, 255, 255)
        cv2.rectangle(frame, (0, 0), (250, 40), (0,0,0), -1)
        cv2.putText(frame, f"AI: {label} {int(conf*100)}%", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, ai_color, 2)
        h, w, _ = frame.shape
        cv2.rectangle(frame, (w-200, 0), (w, 40), (50, 50, 50), -1)
        cv2.putText(frame, f"You:{m_score}  Pi:{p_score}", (w-190, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    def update_frame(frame):
        _, buffer = cv2.imencode('.jpeg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
        image_widget.value = buffer.tobytes()

    # --- 鏡頭翻轉設定 (1=左右鏡像, 0=上下翻轉, -1=上下左右) ---
    FLIP_MODE = 1 

    try:
        with Picamera2() as picam2:
            config = picam2.create_preview_configuration(main={"size": (640, 480)})
            picam2.configure(config)
            picam2.start()
            
            show_on_oled(oled, "Ready...", f"You:{my_score} Pi:{pi_score}")
            
            while True: 
                # 1. 抓圖與辨識
                frame = picam2.capture_array()
                frame = cv2.flip(cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR), FLIP_MODE)
                label, conf = predict_gesture(frame, interpreter, in_idx, out_idx, labels)
                
                # 繪製介面
                draw_hud(frame, label, conf, my_score, pi_score)
                
                # --- [挑戰 A] 判斷觸發 ---
                if is_valid_gesture(______, ______) == True:
                    print("🚀 觸發倒數！")
                    
                    # --- [挑戰 B] 倒數迴圈 ---
                    for i in range(______, ______, -1):
                        
                        # --- [挑戰 C] 使用 OLED 顯示倒數 ---
                        show_on_oled(oled, "Count Down:", f"   {i}   ")
                        
                        # 動態倒數動畫
                        start = time.time()
                        while time.time() - start < 1:
                            tmp_frame = picam2.capture_array()
                            tmp_frame = cv2.flip(cv2.cvtColor(tmp_frame, cv2.COLOR_RGBA2BGR), FLIP_MODE)
                            # 仍要顯示 HUD
                            l, c = predict_gesture(tmp_frame, interpreter, in_idx, out_idx, labels)
                            draw_hud(tmp_frame, l, c, my_score, pi_score)
                            # 畫大數字
                            cv2.putText(tmp_frame, str(i), (280, 240), cv2.FONT_HERSHEY_SIMPLEX, 5, (0,0,255), 5)
                            update_frame(tmp_frame)
                    
                    # --- 決戰時刻 ---
                    final_frame = picam2.capture_array()
                    final_frame = cv2.flip(cv2.cvtColor(final_frame, cv2.COLOR_RGBA2BGR), FLIP_MODE)
                    player_move, _ = predict_gesture(final_frame, interpreter, in_idx, out_idx, labels)
                    
                    # --- [挑戰 D] 電腦出拳 ---
                    computer_move = random.choice([______, ______, ______])
                    
                    # --- [挑戰 E] 判斷勝負 ---
                    result = get_winner(______, ______)
                    
                    # --- [挑戰 F] 更新分數 ---
                    if result == "You Win!":
                        my_score = ______
                    elif result == "You Lose!":
                        pi_score = ______
                    
                    # --- [挑戰 G] OLED 顯示結果 ---
                    show_on_oled(oled, result, f"You:{my_score} Pi:{pi_score}")
                    print(f"結果: {result}")
                    
                    # --- 視覺特效 ---
                    cv2.rectangle(final_frame, (50, 50), (590, 430), (0,0,0), -1)
                    cv2.putText(final_frame, "Battle Result", (200, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                    cv2.putText(final_frame, f"You: {player_move}", (100, 160), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 3)
                    cv2.putText(final_frame, f"Pi : {computer_move}", (100, 220), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)
                    
                    res_color = (0, 255, 255)
                    if "Win" in result: res_color = (0, 255, 0)
                    if "Lose" in result: res_color = (0, 0, 255)
                    cv2.putText(final_frame, result, (120, 320), cv2.FONT_HERSHEY_SIMPLEX, 2, res_color, 5)
                    cv2.putText(final_frame, f"Score -> {my_score} : {pi_score}", (150, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                    
                    update_frame(final_frame)
                    time.sleep(3) # 暫停看結果
                    
                    # 冷卻時間
                    show_on_oled(oled, "Resetting...", "Wait...")
                    start_cool = time.time()
                    while time.time() - start_cool < 2:
                        cool_frame = picam2.capture_array()
                        cool_frame = cv2.flip(cv2.cvtColor(cool_frame, cv2.COLOR_RGBA2BGR), FLIP_MODE)
                        l, c = predict_gesture(cool_frame, interpreter, in_idx, out_idx, labels)
                        draw_hud(cool_frame, l, c, my_score, pi_score)
                        cv2.putText(cool_frame, "Please Reset Hand", (180, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (200,200,200), 2)
                        update_frame(cool_frame)

                    show_on_oled(oled, "Ready...", f"You:{my_score} Pi:{pi_score}")

                update_frame(frame)

    except KeyboardInterrupt:
        print("停止遊戲")
    except Exception as e:
        print(f"錯誤: {e}")
    finally:
        if oled: oled.clear()

# 執行遊戲
start_game_challenge()

### 💡 解答 (點擊展開)

In [ ]:
def start_game_answer():
    oled, interpreter, labels = init_hardware()
    if not oled: return
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    in_idx, out_idx = input_details[0]['index'], output_details[0]['index']
    
    print("👇 遊戲畫面將顯示在下方 (按上方停止鈕結束) 👇")
    image_widget = widgets.Image(format='jpeg', width=640, height=480)
    display(image_widget)

    my_score = 0
    pi_score = 0

    def draw_hud(frame, label, conf, m_score, p_score):
        ai_color = (0, 255, 0) if conf > 0.85 else (0, 255, 255)
        cv2.rectangle(frame, (0, 0), (250, 40), (0,0,0), -1)
        cv2.putText(frame, f"AI: {label} {int(conf*100)}%", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, ai_color, 2)
        h, w, _ = frame.shape
        cv2.rectangle(frame, (w-200, 0), (w, 40), (50, 50, 50), -1)
        cv2.putText(frame, f"You:{m_score}  Pi:{p_score}", (w-190, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    def update_frame(frame):
        _, buffer = cv2.imencode('.jpeg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
        image_widget.value = buffer.tobytes()

    # --- 定義鏡頭翻轉模式 (1=左右鏡像, 0=上下翻轉, -1=上下左右) ---
    FLIP_MODE = 1 

    try:
        with Picamera2() as picam2:
            config = picam2.create_preview_configuration(main={"size": (640, 480)})
            picam2.configure(config)
            picam2.start()
            show_on_oled(oled, "Ready...", f"You:{my_score} Pi:{pi_score}")
            
            while True: 
                frame = picam2.capture_array()
                frame = cv2.flip(cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR), FLIP_MODE)
                label, conf = predict_gesture(frame, interpreter, in_idx, out_idx, labels)
                draw_hud(frame, label, conf, my_score, pi_score)
                
                if is_valid_gesture(label, conf) == True:
                    print("🚀 觸發倒數！")
                    
                    for i in range(3, 0, -1):
                        show_on_oled(oled, "Count Down:", f"   {i}   ")
                        start = time.time()
                        while time.time() - start < 1:
                            tmp_frame = picam2.capture_array()
                            tmp_frame = cv2.flip(cv2.cvtColor(tmp_frame, cv2.COLOR_RGBA2BGR), FLIP_MODE)
                            l, c = predict_gesture(tmp_frame, interpreter, in_idx, out_idx, labels)
                            draw_hud(tmp_frame, l, c, my_score, pi_score)
                            cv2.putText(tmp_frame, str(i), (280, 240), cv2.FONT_HERSHEY_SIMPLEX, 5, (0,0,255), 5)
                            update_frame(tmp_frame)
                    
                    final_frame = picam2.capture_array()
                    final_frame = cv2.flip(cv2.cvtColor(final_frame, cv2.COLOR_RGBA2BGR), FLIP_MODE)
                    player_move, _ = predict_gesture(final_frame, interpreter, in_idx, out_idx, labels)
                    
                    computer_move = random.choice(["Rock", "Paper", "Scissors"])
                    result = get_winner(player_move, computer_move)
                    
                    if result == "You Win!":
                        my_score = my_score + 1
                    elif result == "You Lose!":
                        pi_score = pi_score + 1
                    
                    show_on_oled(oled, result, f"You:{my_score} Pi:{pi_score}")
                    print(f"結果: {result}")
                    
                    cv2.rectangle(final_frame, (50, 50), (590, 430), (0,0,0), -1)
                    cv2.putText(final_frame, "Battle Result", (200, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                    cv2.putText(final_frame, f"You: {player_move}", (100, 160), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 3)
                    cv2.putText(final_frame, f"Pi : {computer_move}", (100, 220), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)
                    
                    res_color = (0, 255, 255)
                    if "Win" in result: res_color = (0, 255, 0)
                    if "Lose" in result: res_color = (0, 0, 255)
                    cv2.putText(final_frame, result, (120, 320), cv2.FONT_HERSHEY_SIMPLEX, 2, res_color, 5)
                    cv2.putText(final_frame, f"Score -> {my_score} : {pi_score}", (150, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                    
                    update_frame(final_frame)
                    time.sleep(3)
                    
                    show_on_oled(oled, "Resetting...", "Wait...")
                    start_cool = time.time()
                    while time.time() - start_cool < 2:
                        cool_frame = picam2.capture_array()
                        cool_frame = cv2.flip(cv2.cvtColor(cool_frame, cv2.COLOR_RGBA2BGR), FLIP_MODE)
                        l, c = predict_gesture(cool_frame, interpreter, in_idx, out_idx, labels)
                        draw_hud(cool_frame, l, c, my_score, pi_score)
                        cv2.putText(cool_frame, "Please Reset Hand", (180, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (200,200,200), 2)
                        update_frame(cool_frame)

                    show_on_oled(oled, "Ready...", f"You:{my_score} Pi:{pi_score}")

                update_frame(frame)

    except KeyboardInterrupt:
        print("停止遊戲")
    except Exception as e:
        print(f"錯誤: {e}")
    finally:
        if oled: oled.clear()

start_game_answer()